# EQCM

In [ ]:
# -*- coding: utf-8 -*-
import sys
from pathlib import Path

import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from matplotlib import gridspec, ticker
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
# Ensure custom module Path is set before import
sys.path.append(r"D:\CHENG\OneDrive - UAB\ICMAB-Python\Figure")
from colors import tol_cmap, tol_cset  # type: ignore

# 画图的初始设置
plt.style.use(r"D:\CHENG\OneDrive - UAB\ICMAB-Python\Figure\liuchzzyy.mplstyle")
# print(plt.style.available)

# xarray setting
xr.set_options(
    cmap_sequential="viridis",
    cmap_divergent="viridis",
    display_width=150,
)  # viridis, gray

# 颜色设定
colors = tol_cset("vibrant")
# Fallback colors in case tol_cset returns None
colors = list(colors) if colors is not None else ["#b0a3d1", "#8bd0d5", "#a8e0ee", "#c5e1a3", "#ffe48b", "#f5a37d", "#e88db1"]
if r"sunset" not in plt.colormaps():
    cmap = tol_cmap("sunset")
    if isinstance(cmap, LinearSegmentedColormap):
        plt.colormaps.register(cmap)
if r"rainbow_PuRd" not in plt.colormaps():
    cmap = tol_cmap("rainbow_PuRd")
    if isinstance(cmap, LinearSegmentedColormap):
        plt.colormaps.register(cmap)  # 备用 plasma

# 输出的文件夹
path_out = Path(r"C:\Users\chengliu\Desktop\Figure")

# Set math font
mpl.rcParams["mathtext.fontset"] = "custom"
mpl.rcParams["mathtext.rm"] = "Arial"
mpl.rcParams["mathtext.it"] = "Arial:italic"
mpl.rcParams["mathtext.bf"] = "Arial:bold"
mpl.rcParams["mathtext.sf"] = "Arial"
mpl.rcParams["mathtext.tt"] = "Arial"
mpl.rcParams["mathtext.cal"] = "Arial"
mpl.rcParams["mathtext.default"] = "regular"

## 20251110 opEQCM

In [ ]:
path_data = Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\EQCM\Testings\20251110\EMD")

echem_files = list(path_data.glob(r"01-*/*.txt"))
eqcm_files = list(path_data.glob(r"**/*_fundamental.csv"))

In [ ]:
# 读取电化学数据
echem = []
for path_file in echem_files:
    with open(path_file, "r", encoding="latin_1") as file:
        for line in file:
            if line.startswith("Nb header lines"):
                line_skip = int(line.split(":")[1].strip())
                break  # 发现后立即退出循环，提高效率
    if 'OCV' in path_file.name:
        df = pd.read_csv(
            path_file, sep="\t", comment="#", skiprows=line_skip - 1, encoding="latin_1", index_col=None, decimal="."
        ).dropna(axis=1, how="all")
        # # 转换数据格式
        df["Ewe-Ece/V"] = df["Ewe-Ece/V"].apply(
                pd.to_numeric, errors="coerce"
            )
        df["time/s"] = df["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
    else:
        df = pd.read_csv(
            path_file, sep="\t", comment="#", skiprows=line_skip - 1, encoding="latin_1", index_col=None, decimal="."
        ).dropna(axis=1, how="all")
        # 转换数据格式
        df[["Ewe-Ece/V", "<I>/mA"]] = df[["Ewe-Ece/V", "<I>/mA"]].apply(
                pd.to_numeric, errors="coerce"
            )
        df["time/s"] = df["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
    echem.append(df)

echem = pd.concat(echem, ignore_index=True, axis=0)
echem = echem.sort_values(by="time/s").reset_index(drop=True)

# 读取 EQCM 的数据
eqcm = []
for path_file in eqcm_files:
    df = pd.read_csv(
        path_file, sep=",", comment="#", index_col=None, header=0, parse_dates=[[0,1]],
    ).dropna(axis=1, how="all")
    # 转换数据格式
    df[["Relative_time", "Temperature", "Resonance_Frequency", "Dissipation"]] = df[["Relative_time", "Temperature", "Resonance_Frequency", "Dissipation"]].apply(
            pd.to_numeric, errors="coerce"
        )
    df["Date_Time"] = df[["Date_Time"]].apply(pd.to_datetime, format="mixed", errors="coerce")
    eqcm.append(df)

eqcm = pd.concat(eqcm, ignore_index=True, axis=0)
eqcm = eqcm.sort_values(by="Date_Time").reset_index(drop=True).rename(columns={"Date_Time": "time/s"})

In [ ]:
# 就近匹配数据
data = pd.merge_asof(echem, eqcm, on="time/s", direction="nearest", tolerance=pd.Timedelta("1s"))
data = data.dropna(subset=["Resonance_Frequency"]).reset_index(drop=True)
data = data.iloc[:-20, :]

In [ ]:
# 画图
%matplotlib inline
plt.close("all")

fig = plt.figure(figsize=(3.3, 5.0))
gs = gridspec.GridSpec(2, 1, height_ratios=[1, 1], width_ratios=None,  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.3)

ax.plot(data["time/s"], data["Ewe-Ece/V"], color=colors[1], linewidth=1.0, label="Voltage")

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V)", fontsize=11)
ax.set_ylim(0.5, 2.0)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=-0.1))

# ax.set_xlabel("Experimental Time", fontsize=11, labelpad=3)
ax.set_xlim(echem["time/s"].min()-pd.Timedelta(minutes=10), echem["time/s"].max()+pd.Timedelta(minutes=10))
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
# plt.setp(ax.get_xticklabels(), rotation=90, ha="center")
ax.set_xticks([])
ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.7, 0.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax2 = ax.twinx()
ax2.plot(data["time/s"], data["<I>/mA"]*1000, color=colors[3], linewidth=1.0, ls=r'--', label="Current")
ax2.set_ylabel(r"$\mathrm{Current \ (\mu A)}$", fontsize=11)
ax2.set_ylim(-8, 16)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(base=4, offset=0))
ax2.yaxis.set_minor_locator(ticker.MultipleLocator(base=2, offset=0))
ax2.legend(loc="upper left", bbox_to_anchor=(0.4, 0.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

# 图 B
subfig = fig.add_subfigure(gs[1, 0])
ax = subfig.add_axes((0, 0.4, 1, 1))
ax.set_box_aspect(0.3)

# ax.plot(data["time/s"], data["Temperature"], color=colors[1], linewidth=1.0, label="Temperature")
ax.plot(data["time/s"], data["Resonance_Frequency"], color=colors[0], linewidth=1.0, label="Resonance Frequency")
# 设置刻度线等格式
ax.set_ylabel(r"Resonance Frequency (Hz)", fontsize=11)
ax.set_ylim(4994000, 5000000)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=2000, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=1000, offset=0))
# ax.set_xlabel("Experimental Time", fontsize=11, labelpad=3)
ax.set_xlim(echem["time/s"].min()-pd.Timedelta(minutes=10), echem["time/s"].max()+pd.Timedelta(minutes=10))
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.setp(ax.get_xticklabels(), rotation=90, ha="center")
ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.05, 1.25), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax2 = ax.twinx()
ax2.plot(data["time/s"], data["Dissipation"]*1000, color=colors[2], linewidth=1.0, label="Dissipation")
ax2.set_ylabel(r"Dissipation (x1000)", fontsize=11)
ax2.set_ylim(0, 15)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax2.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax2.legend(loc="upper left", bbox_to_anchor=(0.6, 1.25), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)


# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_13_300.tif"),
    pad_inches=0.2,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

## 20251010 - 1mL - Pt - 60 uL - Sep

In [ ]:
path_filelist = list(
    Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\EQCM\Testings\251010-Pt\EMD Deposition1-1mL-Pt-100uL-Sepa").glob(
        r"**\*.txt"
    )
)
path_filelist

In [ ]:
# 读取电化学数据
echem = []
for path_file in path_filelist:
    with open(path_file, "r", encoding="latin_1") as file:
        for line in file:
            if line.startswith("Nb header lines"):
                line_skip = int(line.split(":")[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(
        path_file, sep="\t", comment="#", skiprows=line_skip - 1, encoding="latin_1", index_col=None, decimal="."
    ).dropna(axis=1, how="all")
    # # 转换数据格式
    df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]] = df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]].apply(
            pd.to_numeric, errors="coerce"
        )
    df["time/s"] = df["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
    df["cycle number"] = df["cycle number"].astype(float).astype(np.int16)
    echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2])]
    echem[i] = echem[i].copy()
    echem[i]['Voltage/V'] = echem[i]['Ewe/V'] - echem[i]['Ece/V']

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, height_ratios=None, width_ratios=[1, 1],  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

ax.plot(
    echem[0]["time/s"], echem[0]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[0], label=r'Trial.13.p1',
)
ax.plot(
    echem[1]["time/s"], echem[1]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[1], label=r'Trial.13.p2',
)

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.65, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# ax.set_xlabel("Experimental Time", fontsize=11, labelpad=3)
ax.set_xlim(echem[0]["time/s"].min()-pd.Timedelta(minutes=10), echem[1]["time/s"].max()+pd.Timedelta(minutes=10))  # type: ignore
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.setp(ax.get_xticklabels(), rotation=90, ha="center")

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.1, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.15,
    r"$\mathrm{1 \ mL, \ 1 \ M \ Zn(AC)_2+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.03,
    0.07,
    r"$\mathrm{60 \ \mu L, \ 1 \ M \ Zn(SO)_4+0.2 \ M \ Mn(SO)_4 + Separator}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)


ax1 = ax.twinx()
ax1.plot(
    echem[1]["time/s"], echem[1]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[1], label=None,
    ls = "--",
)
ax1.plot(
    echem[0]["time/s"], echem[0]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[0], label=None,
    ls = "--",
)

# 设置刻度线等格式
ax1.set_ylabel(r"Current ($\mathrm{\mu A}$)", fontsize=11)
ax1.set_ylim(-15, 15)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax1.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_13_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

### 局部放大图

In [ ]:
min_index = echem[1][echem[1]['cycle number']==0]["Voltage/V"].idxmin()

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, height_ratios=None, width_ratios=[1, 1],  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

ax.plot(
    echem[0]["time/s"], echem[0]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[0], label=r'Trial.13.p1',
)
ax.plot(
    echem[1]["time/s"][: min_index+1], echem[1]["Voltage/V"][: min_index+1],
    marker="o", markersize=2, linewidth=0.5, color=colors[1], label=r'Trial.13.p2',
)

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.65, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

ax.set_xlabel("Experimental Time", fontsize=11, labelpad=3)
ax.set_xlim(echem[0]["time/s"].min()-pd.Timedelta(minutes=10), echem[1]["time/s"][min_index+1]+pd.Timedelta(minutes=10))  # type: ignore
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.setp(ax.get_xticklabels(), rotation=90, ha="center")

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.6, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.15,
    r"$\mathrm{1 \ mL, \ 1 \ M \ Zn(AC)_2+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.03,
    0.07,
    r"$\mathrm{60 \ \mu L, \ 1 \ M \ Zn(SO)_4+0.2 \ M \ Mn(SO)_4 + Separator}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)


ax1 = ax.twinx()
ax1.plot(
    echem[0]["time/s"], echem[0]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[0], label=None,
    ls = "--",
)
ax1.plot(
    echem[1]["time/s"][: min_index+1], echem[1]["<I>/mA"][: min_index+1]*1000,
    linewidth=1.0, color=colors[1], label=None,
    ls = "--",
)

# 设置刻度线等格式
ax1.set_ylabel(r"Current ($\mathrm{\mu A}$)", fontsize=11)
ax1.set_ylim(-15, 15)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax1.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_13_02_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

## 20251007 - 1mL - Pt - 60 uL

In [ ]:
path_filelist = list(
    Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\EQCM\Testings\251007-Pt\EMD Deposition1-1mL-Pt-60uL").glob(
        r"**\*.txt"
    )
)
path_filelist

In [ ]:
# 读取电化学数据
echem = []
for path_file in path_filelist:
    with open(path_file, "r", encoding="latin_1") as file:
        for line in file:
            if line.startswith("Nb header lines"):
                line_skip = int(line.split(":")[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(
        path_file, sep="\t", comment="#", skiprows=line_skip - 1, encoding="latin_1", index_col=None, decimal="."
    ).dropna(axis=1, how="all")
    # # 转换数据格式
    df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]] = df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]].apply(
            pd.to_numeric, errors="coerce"
        )
    df["time/s"] = df["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
    df["cycle number"] = df["cycle number"].astype(float).astype(np.int16)
    echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1])]
    echem[i] = echem[i].copy()
    echem[i]['Voltage/V'] = echem[i]['Ewe/V'] - echem[i]['Ece/V']

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, height_ratios=None, width_ratios=[1, 1],  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

ax.plot(
    echem[0]["time/s"], echem[0]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[0], label=r'Trial.12.p1',
)
ax.plot(
    echem[1]["time/s"], echem[1]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[1], label=r'Trial.12.p2',
)

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.65, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# ax.set_xlabel("Experimental Time", fontsize=11, labelpad=3)
ax.set_xlim(echem[0]["time/s"].min()-pd.Timedelta(minutes=10), echem[1]["time/s"].max()+pd.Timedelta(minutes=10))  # type: ignore
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.setp(ax.get_xticklabels(), rotation=90, ha="center")

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.6, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.15,
    r"$\mathrm{1 \ mL, \ 1 \ M \ Zn(AC)_2+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.03,
    0.07,
    r"$\mathrm{60 \ \mu L, \ 1 \ M \ Zn(SO)_4+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)


ax1 = ax.twinx()
ax1.plot(
    echem[1]["time/s"], echem[1]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[1], label=None,
    ls = "--",
)
ax1.plot(
    echem[0]["time/s"], echem[0]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[0], label=None,
    ls = "--",
)

# 设置刻度线等格式
ax1.set_ylabel(r"Current ($\mathrm{\mu A}$)", fontsize=11)
ax1.set_ylim(-15, 15)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax1.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_12_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

### 局部放大图

In [ ]:
min_index = echem[1][echem[1]['cycle number']==0]["Voltage/V"].idxmin()

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, height_ratios=None, width_ratios=[1, 1],  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

ax.plot(
    echem[0]["time/s"], echem[0]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[0], label=r'Trial.12.p1',
)
ax.plot(
    echem[1]["time/s"][: min_index+1], echem[1]["Voltage/V"][: min_index+1],
    marker="o", markersize=2, linewidth=0.5, color=colors[1], label=r'Trial.12.p2',
)

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.65, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

ax.set_xlabel("Experimental Time", fontsize=11, labelpad=3)
ax.set_xlim(echem[0]["time/s"].min()-pd.Timedelta(minutes=10), echem[1]["time/s"][min_index+1]+pd.Timedelta(minutes=10))  # type: ignore
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.setp(ax.get_xticklabels(), rotation=90, ha="center")

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.6, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.15,
    r"$\mathrm{1 \ mL, \ 1 \ M \ Zn(AC)_2+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.03,
    0.07,
    r"$\mathrm{60 \ \mu L, \ 1 \ M \ Zn(SO)_4+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)


ax1 = ax.twinx()
ax1.plot(
    echem[0]["time/s"], echem[0]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[0], label=None,
    ls = "--",
)
ax1.plot(
    echem[1]["time/s"][: min_index+1], echem[1]["<I>/mA"][: min_index+1]*1000,
    linewidth=1.0, color=colors[1], label=None,
    ls = "--",
)

# 设置刻度线等格式
ax1.set_ylabel(r"Current ($\mathrm{\mu A}$)", fontsize=11)
ax1.set_ylim(-15, 15)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax1.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_12_02_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

## 20251004 - 1mL - Pt - Separator - 100 uL

In [ ]:
path_filelist = list(
    Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\EQCM\Testings\251004-Pt\EMD Deposition2-1mL-Pt-100uL-Sepa").glob(
        r"**\*.txt"
    )
)
path_filelist

In [ ]:
# 读取电化学数据
echem = []
for path_file in path_filelist:
    with open(path_file, "r", encoding="latin_1") as file:
        for line in file:
            if line.startswith("Nb header lines"):
                line_skip = int(line.split(":")[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(
        path_file, sep="\t", comment="#", skiprows=line_skip - 1, encoding="latin_1", index_col=None, decimal="."
    ).dropna(axis=1, how="all")
    # # 转换数据格式
    df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]] = df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]].apply(
            pd.to_numeric, errors="coerce"
        )
    df["time/s"] = df["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
    df["cycle number"] = df["cycle number"].astype(float).astype(np.int16)
    echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2])]
    echem[i] = echem[i].copy()
    echem[i]['Voltage/V'] = echem[i]['Ewe/V'] - echem[i]['Ece/V']

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, height_ratios=None, width_ratios=[1, 1],  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

ax.plot(
    echem[0]["time/s"], echem[0]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[0], label=r'Trial.11.p1',
)
ax.plot(
    echem[1]["time/s"], echem[1]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[1], label=r'Trial.11.p2',
)

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.65, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# ax.set_xlabel("Experimental Time", fontsize=11, labelpad=3)
ax.set_xlim(echem[0]["time/s"].min()-pd.Timedelta(minutes=10), echem[1]["time/s"].max()+pd.Timedelta(minutes=10))  # type: ignore
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.setp(ax.get_xticklabels(), rotation=90, ha="center")

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.1, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.15,
    r"$\mathrm{1 \ mL, \ 1 \ M \ Zn(AC)_2+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.03,
    0.07,
    r"$\mathrm{100 \ \mu L, \ 1 \ M \ Zn(SO)_4+0.2 \ M \ Mn(SO)_4} + Separators$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)


ax1 = ax.twinx()
ax1.plot(
    echem[1]["time/s"], echem[1]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[1], label=None,
    ls = "--",
)
ax1.plot(
    echem[0]["time/s"], echem[0]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[0], label=None,
    ls = "--",
)

# 设置刻度线等格式
ax1.set_ylabel(r"Current ($\mathrm{\mu A}$)", fontsize=11)
ax1.set_ylim(-15, 15)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax1.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_11_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

### 局部放大图

In [ ]:
min_index = echem[1][echem[1]['cycle number']==0]["Voltage/V"].idxmin()

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, height_ratios=None, width_ratios=[1, 1],  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

ax.plot(
    echem[0]["time/s"], echem[0]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[0], label=r'Trial.11.p1',
)
ax.plot(
    echem[1]["time/s"][: min_index+1], echem[1]["Voltage/V"][: min_index+1],
    marker="o", markersize=2, linewidth=0.5, color=colors[1], label=r'Trial.11.p2',
)

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.65, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

ax.set_xlabel("Experimental Time", fontsize=11, labelpad=3)
ax.set_xlim(echem[0]["time/s"].min()-pd.Timedelta(minutes=10), echem[1]["time/s"][min_index+1]+pd.Timedelta(minutes=10))  # type: ignore
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.setp(ax.get_xticklabels(), rotation=90, ha="center")

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.6, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.15,
    r"$\mathrm{1 \ mL, \ 1 \ M \ Zn(AC)_2+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.03,
    0.07,
    r"$\mathrm{100 \ \mu L, \ 1 \ M \ Zn(SO)_4+0.2 \ M \ Mn(SO)_4 + Separators}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)


ax1 = ax.twinx()
ax1.plot(
    echem[0]["time/s"], echem[0]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[0], label=None,
    ls = "--",
)
ax1.plot(
    echem[1]["time/s"][: min_index+1], echem[1]["<I>/mA"][: min_index+1]*1000,
    linewidth=1.0, color=colors[1], label=None,
    ls = "--",
)

# 设置刻度线等格式
ax1.set_ylabel(r"Current ($\mathrm{\mu A}$)", fontsize=11)
ax1.set_ylim(-15, 15)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax1.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_11_02_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

## 20251004 - 1mL - Pt - 100 uL

In [ ]:
path_filelist = list(
    Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\EQCM\Testings\251004-Pt\EMD Deposition1-1mL-Pt-100uL").glob(
        r"**\*.txt"
    )
)
path_filelist

In [ ]:
# 读取电化学数据
echem = []
for path_file in path_filelist:
    with open(path_file, "r", encoding="latin_1") as file:
        for line in file:
            if line.startswith("Nb header lines"):
                line_skip = int(line.split(":")[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(
        path_file, sep="\t", comment="#", skiprows=line_skip - 1, encoding="latin_1", index_col=None, decimal="."
    ).dropna(axis=1, how="all")
    # # 转换数据格式
    df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]] = df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]].apply(
            pd.to_numeric, errors="coerce"
        )
    df["time/s"] = df["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
    df["cycle number"] = df["cycle number"].astype(float).astype(np.int16)
    echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1])]
    echem[i] = echem[i].copy()
    echem[i]['Voltage/V'] = echem[i]['Ewe/V'] - echem[i]['Ece/V']

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, height_ratios=None, width_ratios=[1, 1],  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

ax.plot(
    echem[0]["time/s"], echem[0]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[0], label=r'Trial.10.p1',
)
ax.plot(
    echem[1]["time/s"], echem[1]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[1], label=r'Trial.10.p2',
)

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.65, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# ax.set_xlabel("Experimental Time", fontsize=11, labelpad=3)
ax.set_xlim(echem[0]["time/s"].min()-pd.Timedelta(minutes=10), echem[1]["time/s"].max()+pd.Timedelta(minutes=10))  # type: ignore
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.setp(ax.get_xticklabels(), rotation=90, ha="center")

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.6, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.15,
    r"$\mathrm{1 \ mL, \ 1 \ M \ Zn(AC)_2+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.03,
    0.07,
    r"$\mathrm{100 \ \mu L, \ 1 \ M \ Zn(SO)_4+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)


ax1 = ax.twinx()
ax1.plot(
    echem[1]["time/s"], echem[1]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[1], label=None,
    ls = "--",
)
ax1.plot(
    echem[0]["time/s"], echem[0]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[0], label=None,
    ls = "--",
)

# 设置刻度线等格式
ax1.set_ylabel(r"Current ($\mathrm{\mu A}$)", fontsize=11)
ax1.set_ylim(-15, 15)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax1.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_10_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

### 局部放大图

In [ ]:
min_index = echem[1][echem[1]['cycle number']==0]["Voltage/V"].idxmin()

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, height_ratios=None, width_ratios=[1, 1],  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

ax.plot(
    echem[0]["time/s"], echem[0]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[0], label=r'Trial.10.p1',
)
ax.plot(
    echem[1]["time/s"][: min_index+1], echem[1]["Voltage/V"][: min_index+1],
    marker="o", markersize=2, linewidth=0.5, color=colors[1], label=r'Trial.10.p2',
)

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.65, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

ax.set_xlabel("Experimental Time", fontsize=11, labelpad=3)
ax.set_xlim(echem[0]["time/s"].min()-pd.Timedelta(minutes=10), echem[1]["time/s"][min_index+1]+pd.Timedelta(minutes=10))  # type: ignore
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.setp(ax.get_xticklabels(), rotation=90, ha="center")

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.6, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.15,
    r"$\mathrm{1 \ mL, \ 1 \ M \ Zn(AC)_2+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.03,
    0.07,
    r"$\mathrm{100 \ \mu L, \ 1 \ M \ Zn(SO)_4+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)


ax1 = ax.twinx()
ax1.plot(
    echem[0]["time/s"], echem[0]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[0], label=None,
    ls = "--",
)
ax1.plot(
    echem[1]["time/s"][: min_index+1], echem[1]["<I>/mA"][: min_index+1]*1000,
    linewidth=1.0, color=colors[1], label=None,
    ls = "--",
)

# 设置刻度线等格式
ax1.set_ylabel(r"Current ($\mathrm{\mu A}$)", fontsize=11)
ax1.set_ylim(-15, 15)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax1.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_10_02_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

## Summary untill 2025-10-02

In [ ]:
path_filelist = list(
    Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\EQCM\Testings").glob( # noqa: E501, RUF001
        r"**\*.txt"
    )
)
path_filelist

In [ ]:
path_namea, path_nameb = [], []
for path_name in path_filelist:
    if "ZnAC2" in path_name.stem:
      path_namea.append(path_name)
    elif "ZnSO4" in path_name.stem:
      path_nameb.append(path_name)

In [ ]:
# 读取电化学数据
echema = []
for path_file in path_namea:
    with open(path_file, "r", encoding="latin_1") as file:
        for line in file:
            if line.startswith("Nb header lines"):
                line_skip = int(line.split(":")[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(
        path_file, sep="\t", comment="#", skiprows=line_skip - 1, encoding="latin_1", index_col=None, decimal="."
    ).dropna(axis=1, how="all")  # noqa: E501
    # # 转换数据格式
    df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]] = df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]].apply(
            pd.to_numeric, errors="coerce"
        )  # noqa: E501
    df["time/s"] = df["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
    df["cycle number"] = df["cycle number"].astype(float).astype(np.int16)
    echema.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echema)):
    echema[i] = echema[i][echema[i].iloc[:, 0].isin([0,1])]  # noqa: E501
    echema[i]['Voltage/V'] = echema[i]['Ewe/V'] - echema[i]['Ece/V']

In [ ]:
# 读取电化学数据
echemb = []
for path_file in path_nameb:
    with open(path_file, "r", encoding="latin_1") as file:
        for line in file:
            if line.startswith("Nb header lines"):
                line_skip = int(line.split(":")[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(
        path_file, sep="\t", comment="#", skiprows=line_skip - 1, encoding="latin_1", index_col=None, decimal="."
    ).dropna(axis=1, how="all")  # noqa: E501
    # # 转换数据格式
    df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]] = df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]].apply(
            pd.to_numeric, errors="coerce"
        )  # noqa: E501
    df["time/s"] = df["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
    df["cycle number"] = df["cycle number"].astype(float).astype(np.int16)
    echemb.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echemb)):
    echemb[i] = echemb[i][echemb[i].iloc[:, 0].isin([0,1])]  # noqa: E501
    echemb[i]['Voltage/V'] = echemb[i]['Ewe/V'] - echemb[i]['Ece/V']

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, height_ratios=None, width_ratios=[1, 1],  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

for i, data in enumerate(echema):
    data.reset_index(drop=True, inplace=True)
    ax.plot(
        data["Capacity/mA.h"]*1000, data["Voltage/V"],
        linewidth=1.0, color=colors[i%8], marker="o", markersize=2,
        label=f'Trial.{i+1}.p1', alpha=1 - i*0.1,
    )
# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.65, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

ax.set_xlabel(r"Capacity ($\mathrm{\mu A \ h})$", fontsize=11, labelpad=3)
ax.set_xlim(-0.5, 10.0)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=2, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=0))

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.65, 0.7), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.07,
    r"$\mathrm{1 \ M \ Zn(AC)_2+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((0.2, 0, 1, 1))
ax.set_box_aspect(0.8)

for i, data in enumerate(echemb):
    min_index = data[data['cycle number']==0]["Voltage/V"].idxmin()
    ax.plot(
        data["Capacity/mA.h"][:min_index]*1000, data["Voltage/V"][:min_index],
        linewidth=1.0, color=colors[i%8], marker="o", markersize=2,
        label=f'Trial.{i+1}.p1', alpha=1 - i*0.1,
    )
    # ax.plot(
    #     data["Capacity/mA.h"][min_index:]*1000, data["Voltage/V"][min_index:],
    #     linewidth=1.0, color=colors[i%8], marker="o", markersize=2,
    #     label=f'Trial.{i+1}.p1', alpha=1 - i*0.1,
    # )

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.65, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

ax.set_xlabel(r"Capacity ($\mathrm{\mu A \ h})$", fontsize=11, labelpad=3)
ax.set_xlim(-0.5, 15.0)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=2, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=0))

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(1.01, 0.7), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.07,
    r"$\mathrm{1 \ M \ Zn(SO)_4+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_sum_01_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, height_ratios=None, width_ratios=[1, 1],  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

for i, data in enumerate(echema):
    data.reset_index(drop=True, inplace=True)
    ax.plot(
        data["Capacity/mA.h"]*1000, data["Voltage/V"],
        linewidth=1.0, color=colors[i%8], marker="o", markersize=2,
        label=f'Trial.{i+1}.p1', alpha=1 - i*0.1,
    )
# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.65, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

ax.set_xlabel(r"Capacity ($\mathrm{\mu A \ h})$", fontsize=11, labelpad=3)
ax.set_xlim(-0.5, 10.0)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=2, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=0))

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.65, 0.7), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.07,
    r"$\mathrm{1 \ M \ Zn(AC)_2+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((0.2, 0, 1, 1))
ax.set_box_aspect(0.8)

for i, data in enumerate(echemb):
    min_index = data[data['cycle number']==0]["Voltage/V"].idxmin()
    # ax.plot(
    #     data["Capacity/mA.h"][:min_index]*1000, data["Voltage/V"][:min_index],
    #     linewidth=1.0, color=colors[i%8], marker="o", markersize=2,
    #     label=f'Trial.{i+1}.p1', alpha=1 - i*0.1,
    # )
    ax.plot(
        data["Capacity/mA.h"][min_index:]*1000, data["Voltage/V"][min_index:],
        linewidth=1.0, color=colors[i%8], marker="o", markersize=2,
        label=f'Trial.{i+1}.p1', alpha=1 - i*0.1,
    )

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.65, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

ax.set_xlabel(r"Capacity ($\mathrm{\mu A \ h})$", fontsize=11, labelpad=3)
ax.set_xlim(-0.5, 15.0)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=2, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=0))

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(1.01, 0.7), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.07,
    r"$\mathrm{1 \ M \ Zn(SO)_4+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_sum_02_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

## 20251002 - 1mL - Pt@SP91

In [ ]:
path_filelist = list(
    Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\EQCM\Testings\251002-Pt@SP91\EMD Deposition1-1mL-Pt@SP91").glob( # noqa: E501, RUF001
        r"**\*.txt"
    )
)
path_filelist

In [ ]:
# 读取电化学数据
echem = []
for path_file in path_filelist:
    with open(path_file, "r", encoding="latin_1") as file:
        for line in file:
            if line.startswith("Nb header lines"):
                line_skip = int(line.split(":")[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(
        path_file, sep="\t", comment="#", skiprows=line_skip - 1, encoding="latin_1", index_col=None, decimal="."
    ).dropna(axis=1, how="all")  # noqa: E501
    # # 转换数据格式
    df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]] = df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]].apply(
            pd.to_numeric, errors="coerce"
        )  # noqa: E501
    df["time/s"] = df["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
    df["cycle number"] = df["cycle number"].astype(float).astype(np.int16)
    echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1])]  # noqa: E501
    echem[i]['Voltage/V'] = echem[i]['Ewe/V'] - echem[i]['Ece/V']

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, height_ratios=None, width_ratios=[1, 1],  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

ax.plot(
    echem[0]["time/s"], echem[0]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[0], label=r'Trial.9.p1',
)
ax.plot(
    echem[1]["time/s"], echem[1]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[1], label=r'Trial.9.p2',
)

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.65, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# ax.set_xlabel("Experimental Time", fontsize=11, labelpad=3)
ax.set_xlim(echem[0]["time/s"].min()-pd.Timedelta(minutes=10), echem[1]["time/s"].max()+pd.Timedelta(minutes=10))  # type: ignore
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.setp(ax.get_xticklabels(), rotation=90, ha="center")

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.6, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.15,
    r"$\mathrm{1 \ mL, \ 1 \ M \ Zn(AC)_2+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.03,
    0.07,
    r"$\mathrm{1 \ mL, \ 1 \ M \ Zn(SO)_4+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)


ax1 = ax.twinx()
ax1.plot(
    echem[1]["time/s"], echem[1]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[1], label=None,
    ls = "--",
)
ax1.plot(
    echem[0]["time/s"], echem[0]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[0], label=None,
    ls = "--",
)

# 设置刻度线等格式
ax1.set_ylabel(r"Current ($\mathrm{\mu A}$)", fontsize=11)
ax1.set_ylim(-15, 15)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax1.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_09_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

### 局部放大图

In [ ]:
min_index = echem[1][echem[1]['cycle number']==0]["Voltage/V"].idxmin()

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, height_ratios=None, width_ratios=[1, 1],  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

ax.plot(
    echem[0]["time/s"], echem[0]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[0], label=r'Trial.9.p1',
)
ax.plot(
    echem[1]["time/s"][: min_index+1], echem[1]["Voltage/V"][: min_index+1],
    marker="o", markersize=2, linewidth=0.5, color=colors[1], label=r'Trial.9.p2',
)

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.65, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

ax.set_xlabel("Experimental Time", fontsize=11, labelpad=3)
ax.set_xlim(echem[0]["time/s"].min()-pd.Timedelta(minutes=10), echem[1]["time/s"][min_index+1]+pd.Timedelta(minutes=10))  # type: ignore
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.setp(ax.get_xticklabels(), rotation=90, ha="center")

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.6, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.15,
    r"$\mathrm{1 \ mL, \ 1 \ M \ Zn(AC)_2+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.03,
    0.07,
    r"$\mathrm{1 \ mL, \ 1 \ M \ Zn(SO)_4+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)


ax1 = ax.twinx()
ax1.plot(
    echem[0]["time/s"], echem[0]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[0], label=None,
    ls = "--",
)
ax1.plot(
    echem[1]["time/s"][: min_index+1], echem[1]["<I>/mA"][: min_index+1]*1000,
    linewidth=1.0, color=colors[1], label=None,
    ls = "--",
)

# 设置刻度线等格式
ax1.set_ylabel(r"Current ($\mathrm{\mu A}$)", fontsize=11)
ax1.set_ylim(-15, 15)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax1.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_09_02_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

## 20251001 - 2mL - Pt

In [ ]:
path_filelist = list(
    Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\EQCM\Testings\251001\EMD Deposition1-2mL-Pt").glob( # noqa: E501, RUF001
        r"**\*.txt"
    )
)
path_filelist

In [ ]:
# 读取电化学数据
echem = []
for path_file in path_filelist:
    with open(path_file, "r", encoding="latin_1") as file:
        for line in file:
            if line.startswith("Nb header lines"):
                line_skip = int(line.split(":")[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(
        path_file, sep="\t", comment="#", skiprows=line_skip - 1, encoding="latin_1", index_col=None, decimal="."
    ).dropna(axis=1, how="all")  # noqa: E501
    # # 转换数据格式
    df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]] = df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]].apply(
            pd.to_numeric, errors="coerce"
        )  # noqa: E501
    df["time/s"] = df["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
    df["cycle number"] = df["cycle number"].astype(float).astype(np.int16)
    echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1])]  # noqa: E501
    echem[i]['Voltage/V'] = echem[i]['Ewe/V'] - echem[i]['Ece/V']

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, height_ratios=None, width_ratios=[1, 1],  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

ax.plot(
    echem[0]["time/s"], echem[0]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[0], label=r'Trial.8.p1',
)
ax.plot(
    echem[1]["time/s"], echem[1]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[1], label=r'Trial.8.p2',
)

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.65, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# ax.set_xlabel("Experimental Time", fontsize=11, labelpad=3)
ax.set_xlim(echem[0]["time/s"].min()-pd.Timedelta(minutes=10), echem[1]["time/s"].max()+pd.Timedelta(minutes=10))  # type: ignore
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.setp(ax.get_xticklabels(), rotation=90, ha="center")

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.6, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.15,
    r"$\mathrm{2 \ mL, \ 1 \ M \ Zn(AC)_2+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.03,
    0.07,
    r"$\mathrm{2 \ mL, \ 1 \ M \ Zn(SO)_4+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)


ax1 = ax.twinx()
ax1.plot(
    echem[1]["time/s"], echem[1]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[1], label=None,
    ls = "--",
)
ax1.plot(
    echem[0]["time/s"], echem[0]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[0], label=None,
    ls = "--",
)

# 设置刻度线等格式
ax1.set_ylabel(r"Current ($\mathrm{\mu A}$)", fontsize=11)
ax1.set_ylim(-15, 15)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax1.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_08_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

### 局部放大图

In [ ]:
min_index = echem[1][echem[1]['cycle number']==0]["Voltage/V"].idxmin()

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, height_ratios=None, width_ratios=[1, 1],  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

ax.plot(
    echem[0]["time/s"], echem[0]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[0], label=r'Trial.8.p1',
)
ax.plot(
    echem[1]["time/s"][: min_index+1], echem[1]["Voltage/V"][: min_index+1],
    marker="o", markersize=2, linewidth=0.5, color=colors[1], label=r'Trial.8.p2',
)

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.65, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

ax.set_xlabel("Experimental Time", fontsize=11, labelpad=3)
ax.set_xlim(echem[0]["time/s"].min()-pd.Timedelta(minutes=10), echem[1]["time/s"][min_index+1]+pd.Timedelta(minutes=10))  # type: ignore
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.setp(ax.get_xticklabels(), rotation=90, ha="center")

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.6, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.15,
    r"$\mathrm{2 \ mL, \ 1 \ M \ Zn(AC)_2+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.03,
    0.07,
    r"$\mathrm{2 \ mL, \ 1 \ M \ Zn(SO)_4+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)


ax1 = ax.twinx()
ax1.plot(
    echem[0]["time/s"], echem[0]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[0], label=None,
    ls = "--",
)
ax1.plot(
    echem[1]["time/s"][: min_index+1], echem[1]["<I>/mA"][: min_index+1]*1000,
    linewidth=1.0, color=colors[1], label=None,
    ls = "--",
)

# 设置刻度线等格式
ax1.set_ylabel(r"Current ($\mathrm{\mu A}$)", fontsize=11)
ax1.set_ylim(-15, 15)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax1.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_08_02_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

## 20250930 - 2mL - CP

In [ ]:
path_filelist = list(
    Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\EQCM\Testings\250930\EMD Deposition1-2mL-CP").glob( # noqa: E501, RUF001
        r"**\*.txt"
    )
)
path_filelist

In [ ]:
# 读取电化学数据
echem = []
for path_file in path_filelist:
    with open(path_file, "r", encoding="latin_1") as file:
        for line in file:
            if line.startswith("Nb header lines"):
                line_skip = int(line.split(":")[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(
        path_file, sep="\t", comment="#", skiprows=line_skip - 1, encoding="latin_1", index_col=None, decimal="."
    ).dropna(axis=1, how="all")  # noqa: E501
    # # 转换数据格式
    df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]] = df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]].apply(
            pd.to_numeric, errors="coerce"
        )  # noqa: E501
    df["time/s"] = df["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
    df["cycle number"] = df["cycle number"].astype(float).astype(np.int16)
    echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1])]  # noqa: E501
    echem[i]['Voltage/V'] = echem[i]['Ewe/V'] - echem[i]['Ece/V']

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, height_ratios=None, width_ratios=[1, 1],  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

ax.plot(
    echem[0]["time/s"], echem[0]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[0], label=r'Trial.7.p1',
)
ax.plot(
    echem[1]["time/s"], echem[1]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[1], label=r'Trial.7.p2',
)

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.65, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# ax.set_xlabel("Experimental Time", fontsize=11, labelpad=3)
ax.set_xlim(echem[0]["time/s"].min()-pd.Timedelta(minutes=10), echem[1]["time/s"].max()+pd.Timedelta(minutes=10))  # type: ignore
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.setp(ax.get_xticklabels(), rotation=90, ha="center")

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.6, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.15,
    r"$\mathrm{2 \ mL, \ 1 \ M \ Zn(AC)_2+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.03,
    0.07,
    r"$\mathrm{2 \ mL, \ 1 \ M \ Zn(SO)_4+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)


ax1 = ax.twinx()
ax1.plot(
    echem[1]["time/s"], echem[1]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[1], label=None,
    ls = "--",
)
ax1.plot(
    echem[0]["time/s"], echem[0]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[0], label=None,
    ls = "--",
)

# 设置刻度线等格式
ax1.set_ylabel(r"Current ($\mathrm{\mu A}$)", fontsize=11)
ax1.set_ylim(-15, 15)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax1.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_07_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

### 局部放大图

In [ ]:
min_index = echem[1][echem[1]['cycle number']==0]["Voltage/V"].idxmin()

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, height_ratios=None, width_ratios=[1, 1],  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

# ax.plot(
#     echem[0]["time/s"], echem[0]["Voltage/V"],
#     marker="o", markersize=2, linewidth=0.5, color=colors[0], label=r'Trial.7.p1',
# )
ax.plot(
    echem[1]["time/s"][: min_index+1], echem[1]["Voltage/V"][: min_index+1],
    marker="o", markersize=2, linewidth=0.5, color=colors[1], label=r'Trial.7.p2',
)

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.65, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# ax.set_xlabel("Experimental Time", fontsize=11, labelpad=3)
# ax.set_xlim(echem[0]["time/s"].min()-pd.Timedelta(minutes=10), echem[1]["time/s"][min_index+1]+pd.Timedelta(minutes=10))  # type: ignore
ax.set_xlim(echem[1]["time/s"].min()-pd.Timedelta(minutes=10), echem[1]["time/s"][min_index+1]+pd.Timedelta(minutes=10))  # type: ignore
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.setp(ax.get_xticklabels(), rotation=90, ha="center")

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.6, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.15,
    r"$\mathrm{2 \ mL, \ 1 \ M \ Zn(AC)_2+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.03,
    0.07,
    r"$\mathrm{2 \ mL, \ 1 \ M \ Zn(SO)_4+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)


ax1 = ax.twinx()
# ax1.plot(
#     echem[0]["time/s"], echem[0]["<I>/mA"]*1000,
#     linewidth=1.0, color=colors[0], label=None,
#     ls = "--",
# )

ax1.plot(
    echem[1]["time/s"][: min_index+1], echem[1]["<I>/mA"][: min_index+1]*1000,
    linewidth=1.0, color=colors[1], label=None,
    ls = "--",
)

# 设置刻度线等格式
ax1.set_ylabel(r"Current ($\mathrm{\mu A}$)", fontsize=11)
ax1.set_ylim(-15, 15)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax1.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_07_02_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

## 20250929 - 2mL - Pt - YG

In [ ]:
path_filelist = list(
    Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\EQCM\Testings\250929YG\EMD Deposition1-2mL-Pt").glob( # noqa: E501, RUF001
        r"**\*.txt"
    )
)
path_filelist

In [ ]:
# 读取电化学数据
echem = []
for path_file in path_filelist:
    with open(path_file, "r", encoding="latin_1") as file:
        for line in file:
            if line.startswith("Nb header lines"):
                line_skip = int(line.split(":")[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(
        path_file, sep="\t", comment="#", skiprows=line_skip - 1, encoding="latin_1", index_col=None, decimal="."
    ).dropna(axis=1, how="all")  # noqa: E501
    # # 转换数据格式
    df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]] = df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]].apply(
            pd.to_numeric, errors="coerce"
        )  # noqa: E501
    df["time/s"] = df["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
    df["cycle number"] = df["cycle number"].astype(float).astype(np.int16)
    echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1])]  # noqa: E501
    echem[i]['Voltage/V'] = echem[i]['Ewe/V'] - echem[i]['Ece/V']

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, height_ratios=None, width_ratios=[1, 1],  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

ax.plot(
    echem[0]["time/s"], echem[0]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[0], label=r'Trial.6.p1',
)
ax.plot(
    echem[1]["time/s"], echem[1]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[1], label=r'Trial.6.p2',
)

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.65, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# ax.set_xlabel("Experimental Time", fontsize=11, labelpad=3)
ax.set_xlim(echem[0]["time/s"].min()-pd.Timedelta(minutes=10), echem[1]["time/s"].max()+pd.Timedelta(minutes=10))  # type: ignore
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.setp(ax.get_xticklabels(), rotation=90, ha="center")

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.6, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.15,
    r"$\mathrm{2 \ mL, \ 1 \ M \ Zn(AC)_2+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.03,
    0.07,
    r"$\mathrm{2 \ mL, \ 1 \ M \ Zn(SO)_4+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)


ax1 = ax.twinx()
ax1.plot(
    echem[1]["time/s"], echem[1]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[1], label=None,
    ls = "--",
)
ax1.plot(
    echem[0]["time/s"], echem[0]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[0], label=None,
    ls = "--",
)

# 设置刻度线等格式
ax1.set_ylabel(r"Current ($\mathrm{\mu A}$)", fontsize=11)
ax1.set_ylim(-15, 15)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax1.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_06_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

### 局部放大图

In [ ]:
min_index = echem[1][echem[1]['cycle number']==0]["Voltage/V"].idxmin()

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, height_ratios=None, width_ratios=[1, 1],  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

ax.plot(
    echem[0]["time/s"], echem[0]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[0], label=r'Trial.6.p1',
)
ax.plot(
    echem[1]["time/s"][: min_index+1], echem[1]["Voltage/V"][: min_index+1],
    marker="o", markersize=2, linewidth=0.5, color=colors[1], label=r'Trial.6.p2',
)

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.65, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# ax.set_xlabel("Experimental Time", fontsize=11, labelpad=3)
ax.set_xlim(echem[0]["time/s"].min()-pd.Timedelta(minutes=10), echem[1]["time/s"][min_index+1]+pd.Timedelta(minutes=10))  # type: ignore
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.setp(ax.get_xticklabels(), rotation=90, ha="center")

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.6, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.15,
    r"$\mathrm{2 \ mL, \ 1 \ M \ Zn(AC)_2+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.03,
    0.07,
    r"$\mathrm{2 \ mL, \ 1 \ M \ Zn(SO)_4+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)


ax1 = ax.twinx()
ax1.plot(
    echem[1]["time/s"][: min_index+1], echem[1]["<I>/mA"][: min_index+1]*1000,
    linewidth=1.0, color=colors[1], label=None,
    ls = "--",
)
ax1.plot(
    echem[0]["time/s"], echem[0]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[0], label=None,
    ls = "--",
)

# 设置刻度线等格式
ax1.set_ylabel(r"Current ($\mathrm{\mu A}$)", fontsize=11)
ax1.set_ylim(-15, 15)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax1.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_06_02_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

## 20250926 - 2 - CP

In [ ]:
path_filelist = list(
    Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\EQCM\Testings\250926\EMD Deposition2-2mL-CP").glob( # noqa: E501, RUF001
        r"**\*.txt"
    )
)
path_filelist

In [ ]:
# 读取电化学数据
echem = []
for path_file in path_filelist:
    with open(path_file, "r", encoding="latin_1") as file:
        for line in file:
            if line.startswith("Nb header lines"):
                line_skip = int(line.split(":")[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(
        path_file, sep="\t", comment="#", skiprows=line_skip - 1, encoding="latin_1", index_col=None, decimal="."
    ).dropna(axis=1, how="all")  # noqa: E501
    # # 转换数据格式
    df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]] = df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]].apply(
            pd.to_numeric, errors="coerce"
        )  # noqa: E501
    df["time/s"] = df["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
    df["cycle number"] = df["cycle number"].astype(float).astype(np.int16)
    echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1])]  # noqa: E501
    echem[i]['Voltage/V'] = echem[i]['Ewe/V'] - echem[i]['Ece/V']

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, height_ratios=None, width_ratios=[1, 1],  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

ax.plot(
    echem[0]["time/s"], echem[0]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[0], label=r'Trial.5.p1',
)
ax.plot(
    echem[1]["time/s"], echem[1]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[1], label=r'Trial.5.p2',
)

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.65, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# ax.set_xlabel("Experimental Time", fontsize=11, labelpad=3)
ax.set_xlim(echem[0]["time/s"].min()-pd.Timedelta(minutes=10), echem[1]["time/s"].max()+pd.Timedelta(minutes=10))  # type: ignore
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.setp(ax.get_xticklabels(), rotation=90, ha="center")

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.6, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.15,
    r"$\mathrm{2 \ mL, \ 1 \ M \ Zn(AC)_2+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.03,
    0.07,
    r"$\mathrm{2 \ mL, \ 1 \ M \ Zn(SO)_4+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)


ax1 = ax.twinx()
ax1.plot(
    echem[1]["time/s"], echem[1]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[1], label=None,
    ls = "--",
)
ax1.plot(
    echem[0]["time/s"], echem[0]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[0], label=None,
    ls = "--",
)

# 设置刻度线等格式
ax1.set_ylabel(r"Current ($\mathrm{\mu A}$)", fontsize=11)
ax1.set_ylim(-15, 15)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax1.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_05_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

### 局部放大图

In [ ]:
min_index = echem[1][echem[1]['cycle number']==0]["Voltage/V"].idxmin()

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, height_ratios=None, width_ratios=[1, 1],  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

ax.plot(
    echem[0]["time/s"], echem[0]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[0], label=r'Trial.5.p1',
)
ax.plot(
    echem[1]["time/s"][: min_index+1], echem[1]["Voltage/V"][: min_index+1],
    marker="o", markersize=2, linewidth=0.5, color=colors[1], label=r'Trial.5.p2',
)

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.65, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# ax.set_xlabel("Experimental Time", fontsize=11, labelpad=3)
ax.set_xlim(echem[0]["time/s"].min()-pd.Timedelta(minutes=10), echem[1]["time/s"][min_index+1]+pd.Timedelta(minutes=10))  # type: ignore
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.setp(ax.get_xticklabels(), rotation=90, ha="center")

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.6, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.15,
    r"$\mathrm{2 \ mL, \ 1 \ M \ Zn(AC)_2+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.03,
    0.07,
    r"$\mathrm{2 \ mL, \ 1 \ M \ Zn(SO)_4+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)


ax1 = ax.twinx()
ax1.plot(
    echem[1]["time/s"][: min_index+1], echem[1]["<I>/mA"][: min_index+1]*1000,
    linewidth=1.0, color=colors[1], label=None,
    ls = "--",
)
ax1.plot(
    echem[0]["time/s"], echem[0]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[0], label=None,
    ls = "--",
)

# 设置刻度线等格式
ax1.set_ylabel(r"Current ($\mathrm{\mu A}$)", fontsize=11)
ax1.set_ylim(-15, 15)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax1.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_05_02_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

## 20250926 - 1

In [ ]:
path_filelist = list(
    Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\EQCM\Testings\250926\EMD Deposition1-2mL").glob( # noqa: E501, RUF001
        r"**\*.txt"
    )
)
path_filelist

In [ ]:
# 读取电化学数据
echem = []
for path_file in path_filelist:
    with open(path_file, "r", encoding="latin_1") as file:
        for line in file:
            if line.startswith("Nb header lines"):
                line_skip = int(line.split(":")[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(
        path_file, sep="\t", comment="#", skiprows=line_skip - 1, encoding="latin_1", index_col=None, decimal="."
    ).dropna(axis=1, how="all")  # noqa: E501
    # # 转换数据格式
    df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]] = df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]].apply(
            pd.to_numeric, errors="coerce"
        )  # noqa: E501
    df["time/s"] = df["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
    df["cycle number"] = df["cycle number"].astype(float).astype(np.int16)
    echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1])]  # noqa: E501
    echem[i]['Voltage/V'] = echem[i]['Ewe/V'] - echem[i]['Ece/V']

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, height_ratios=None, width_ratios=[1, 1],  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

ax.plot(
    echem[0]["time/s"], echem[0]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[0], label=r'Trial.4.p1',
)
ax.plot(
    echem[1]["time/s"], echem[1]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[1], label=r'Trial.4.p2',
)

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.65, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# ax.set_xlabel("Experimental Time", fontsize=11, labelpad=3)
ax.set_xlim(echem[0]["time/s"].min()-pd.Timedelta(minutes=10), echem[1]["time/s"].max()+pd.Timedelta(minutes=10))  # type: ignore
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.setp(ax.get_xticklabels(), rotation=90, ha="center")

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.3, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.15,
    r"$\mathrm{2 \ mL, \ 1 \ M \ Zn(AC)_2+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.03,
    0.07,
    r"$\mathrm{2 \ mL, \ 1 \ M \ Zn(SO)_4+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)


ax1 = ax.twinx()
ax1.plot(
    echem[1]["time/s"], echem[1]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[1], label=None,
    ls = "--",
)
ax1.plot(
    echem[0]["time/s"], echem[0]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[0], label=None,
    ls = "--",
)

# 设置刻度线等格式
ax1.set_ylabel(r"Current ($\mathrm{\mu A}$)", fontsize=11)
ax1.set_ylim(-15, 15)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax1.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_04_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

### 局部放大图

In [ ]:
min_index = echem[1]["Voltage/V"].idxmin()

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, height_ratios=None, width_ratios=[1, 1],  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

ax.plot(
    echem[0]["time/s"], echem[0]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[0], label=r'Trial.4.p1',
)
ax.plot(
    echem[1]["time/s"][: min_index+1], echem[1]["Voltage/V"][: min_index+1],
    marker="o", markersize=2, linewidth=0.5, color=colors[1], label=r'Trial.4.p2',
)

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.65, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# ax.set_xlabel("Experimental Time", fontsize=11, labelpad=3)
ax.set_xlim(echem[0]["time/s"].min()-pd.Timedelta(minutes=10), echem[1]["time/s"][min_index+1]+pd.Timedelta(minutes=10))  # type: ignore
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.setp(ax.get_xticklabels(), rotation=90, ha="center")

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.6, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.15,
    r"$\mathrm{2 \ mL, \ 1 \ M \ Zn(AC)_2+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.03,
    0.07,
    r"$\mathrm{2 \ mL, \ 1 \ M \ Zn(SO)_4+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)


ax1 = ax.twinx()
ax1.plot(
    echem[1]["time/s"][: min_index+1], echem[1]["<I>/mA"][: min_index+1]*1000,
    linewidth=1.0, color=colors[1], label=None,
    ls = "--",
)
ax1.plot(
    echem[0]["time/s"], echem[0]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[0], label=None,
    ls = "--",
)

# 设置刻度线等格式
ax1.set_ylabel(r"Current ($\mathrm{\mu A}$)", fontsize=11)
ax1.set_ylim(-15, 15)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax1.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_04_02_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

## 20250925

In [ ]:
path_filelist = list(
    Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\EQCM\Testings\250925\EMD Deposition1").glob( # noqa: E501, RUF001
        r"**\*.txt"
    )
)
path_filelist

In [ ]:
# 读取电化学数据
echem = []
for path_file in path_filelist:
    with open(path_file, "r", encoding="latin_1") as file:
        for line in file:
            if line.startswith("Nb header lines"):
                line_skip = int(line.split(":")[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(
        path_file, sep="\t", comment="#", skiprows=line_skip - 1, encoding="latin_1", index_col=None, decimal="."
    ).dropna(axis=1, how="all")  # noqa: E501
    # # 转换数据格式
    df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]] = df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]].apply(
            pd.to_numeric, errors="coerce"
        )  # noqa: E501
    df["time/s"] = df["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
    df["cycle number"] = df["cycle number"].astype(float).astype(np.int16)
    echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1])]  # noqa: E501
    echem[i]['Voltage/V'] = echem[i]['Ewe/V'] - echem[i]['Ece/V']

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, height_ratios=None, width_ratios=[1, 1],  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

ax.plot(
    echem[0]["time/s"], echem[0]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[0], label=r'Trial.3.p1',
)
ax.plot(
    echem[1]["time/s"], echem[1]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[1], label=r'Trial.3.p2',
)

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.65, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# ax.set_xlabel("Experimental Time", fontsize=11, labelpad=3)
ax.set_xlim(echem[0]["time/s"].min()-pd.Timedelta(minutes=10), echem[1]["time/s"].max()+pd.Timedelta(minutes=10))  # type: ignore
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.setp(ax.get_xticklabels(), rotation=90, ha="center")

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.6, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.15,
    r"$\mathrm{4 \ mL, \ 1 \ M \ Zn(AC)_2+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.03,
    0.07,
    r"$\mathrm{4 \ mL, \ 1 \ M \ Zn(SO)_4+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)


ax1 = ax.twinx()
ax1.plot(
    echem[1]["time/s"], echem[1]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[1], label=None,
    ls = "--",
)
ax1.plot(
    echem[0]["time/s"], echem[0]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[0], label=None,
    ls = "--",
)

# 设置刻度线等格式
ax1.set_ylabel(r"Current ($\mathrm{\mu A}$)", fontsize=11)
ax1.set_ylim(-15, 15)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax1.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_03_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

### 局部放大图

In [ ]:
min_index = echem[1]["Voltage/V"].idxmin()

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, height_ratios=None, width_ratios=[1, 1],  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

ax.plot(
    echem[0]["time/s"], echem[0]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[0], label=r'Trial.3.p1',
)
ax.plot(
    echem[1]["time/s"][: min_index+1], echem[1]["Voltage/V"][: min_index+1],
    marker="o", markersize=2, linewidth=0.5, color=colors[1], label=r'Trial.3.p2',
)

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.65, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# ax.set_xlabel("Experimental Time", fontsize=11, labelpad=3)
ax.set_xlim(echem[0]["time/s"].min()-pd.Timedelta(minutes=10), echem[1]["time/s"][min_index+1]+pd.Timedelta(minutes=10))  # type: ignore
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.setp(ax.get_xticklabels(), rotation=90, ha="center")

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.6, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.15,
    r"$\mathrm{4 \ mL, \ 1 \ M \ Zn(AC)_2+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.03,
    0.07,
    r"$\mathrm{4 \ mL, \ 1 \ M \ Zn(SO)_4+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)


ax1 = ax.twinx()
ax1.plot(
    echem[1]["time/s"][: min_index+1], echem[1]["<I>/mA"][: min_index+1]*1000,
    linewidth=1.0, color=colors[1], label=None,
    ls = "--",
)
ax1.plot(
    echem[0]["time/s"], echem[0]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[0], label=None,
    ls = "--",
)

# 设置刻度线等格式
ax1.set_ylabel(r"Current ($\mathrm{\mu A}$)", fontsize=11)
ax1.set_ylim(-15, 15)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax1.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_03_02_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

## 20250918 - 02

In [ ]:
path_filelist = list(
    Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\EQCM\Testings\250918\EMD Deposition2").glob( # noqa: E501, RUF001
        r"**\*.txt"
    )
)
path_filelist

In [ ]:
# 读取电化学数据
echem = []
for path_file in path_filelist:
    with open(path_file, "r", encoding="latin_1") as file:
        for line in file:
            if line.startswith("Nb header lines"):
                line_skip = int(line.split(":")[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(
        path_file, sep="\t", comment="#", skiprows=line_skip - 1, encoding="latin_1", index_col=None, decimal="."
    ).dropna(axis=1, how="all")  # noqa: E501
    # # 转换数据格式
    df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]] = df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]].apply(
            pd.to_numeric, errors="coerce"
        )  # noqa: E501
    df["time/s"] = df["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
    df["cycle number"] = df["cycle number"].astype(float).astype(np.int16)
    echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1])]  # noqa: E501
    echem[i]['Voltage/V'] = echem[i]['Ewe/V'] - echem[i]['Ece/V']

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, height_ratios=None, width_ratios=[1, 1],  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

ax.plot(
    echem[0]["time/s"], echem[0]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[0], label=r'Trial.2.p1',
)
ax.plot(
    echem[1]["time/s"], echem[1]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[1], label=r'Trial.2.p2',
)

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(1.05, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# ax.set_xlabel("Experimental Time", fontsize=11, labelpad=3)
ax.set_xlim(echem[0]["time/s"].min()-pd.Timedelta(minutes=10), echem[1]["time/s"].max()+pd.Timedelta(minutes=10))  # type: ignore
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.setp(ax.get_xticklabels(), rotation=90, ha="center")

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.6, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.15,
    r"$\mathrm{4 \ mL, \ 1 \ M \ Zn(AC)_2+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.03,
    0.07,
    r"$\mathrm{4 \ mL, \ 1 \ M \ Zn(SO)_4+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)


ax1 = ax.twinx()
ax1.plot(
    echem[1]["time/s"], echem[1]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[1], label=None,
    ls = "--",
)
ax1.plot(
    echem[0]["time/s"], echem[0]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[0], label=None,
    ls = "--",
)

# 设置刻度线等格式
ax1.set_ylabel(r"Current ($\mathrm{\mu A}$)", fontsize=11)
ax1.set_ylim(-15, 15)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax1.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_02_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

## 20250918 - 01

In [ ]:
path_filelist = list(
    Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\EQCM\Testings\250918\EMD Deposition1").glob( # noqa: E501, RUF001
        r"**\*.txt"
    )
)
path_filelist

In [ ]:
# 读取电化学数据
echem = []
for path_file in path_filelist:
    with open(path_file, "r", encoding="latin_1") as file:
        for line in file:
            if line.startswith("Nb header lines"):
                line_skip = int(line.split(":")[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(
        path_file, sep="\t", comment="#", skiprows=line_skip - 1, encoding="latin_1", index_col=None, decimal="."
    ).dropna(axis=1, how="all")  # noqa: E501
    # # 转换数据格式
    df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]] = df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]].apply(
            pd.to_numeric, errors="coerce"
        )  # noqa: E501
    df["time/s"] = df["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
    df["cycle number"] = df["cycle number"].astype(float).astype(np.int16)
    echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1])]  # noqa: E501
    echem[i]['Voltage/V'] = echem[i]['Ewe/V'] - echem[i]['Ece/V']

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, height_ratios=None, width_ratios=[1, 1],  wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

ax.plot(
    echem[0]["time/s"], echem[0]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[0], label=r'Trial.1.p1',
)
ax.plot(
    echem[1]["time/s"], echem[1]["Voltage/V"],
    marker="o", markersize=2, linewidth=0.5, color=colors[1], label=r'Trial.1.p2',
)

# 设置刻度线等格式
ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.45, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# ax.set_xlabel("Experimental Time", fontsize=11, labelpad=3)
ax.set_xlim(echem[0]["time/s"].min()-pd.Timedelta(minutes=10), echem[1]["time/s"].max()+pd.Timedelta(minutes=30))  # type: ignore
ax.xaxis.set_major_locator(mdates.HourLocator(interval=2))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=60))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.setp(ax.get_xticklabels(), rotation=90, ha="center")

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.6, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.03,
    0.15,
    r"$\mathrm{4 \ mL, \ 1 \ M \ Zn(AC)_2+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.03,
    0.07,
    r"$\mathrm{4 \ mL, \ 1 \ M \ Zn(SO)_4+0.2 \ M \ Mn(SO)_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)


ax1 = ax.twinx()
ax1.plot(
    echem[1]["time/s"], echem[1]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[1], label=None,
    ls = "--",
)
ax1.plot(
    echem[0]["time/s"], echem[0]["<I>/mA"]*1000,
    linewidth=1.0, color=colors[0], label=None,
    ls = "--",
)

# 设置刻度线等格式
ax1.set_ylabel(r"Current ($\mathrm{\mu A}$)", fontsize=11)
ax1.set_ylim(-10, 10)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(base=4, offset=2))
ax1.yaxis.set_minor_locator(ticker.MultipleLocator(base=2, offset=2))

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"EQCM_Testing_01_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()